# Analyzing Wanderers

Important: This notebook uses [Plotly](https://plotly.com/python/) mostly to visualize data. Plotly charts won't be rendered in Githubs notebook viewer. So please [view this notebook on nbviewer](https://nbviewer.org/github/ymyke/abalyzer/blob/main/fidenzas.ipynb) (or on your local installation after cloning the repo). Note that nbviewer caches notebooks and sometimes takes a long time to pick up a new version.

On the upside, all the Plotly charts are interactive. You can zoom, hover, pan, etc. Zooming is especially important due to the outliers in some the data so you can zoom in on the interesting parts.

In [1]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"

## Get the assets

In [2]:
import datetime
from opensea import retrieve_assets

# ids = list(range(0, 8888))
ids = list(range(0, 1000)) # FIXME
assets = retrieve_assets(token_ids=ids, contract="0x8184a482A5038B124d933B779E0Ea6e0fb72F54E")
print(datetime.datetime.now())


.................................................. -- All 1000 assets retrieved.
2021-10-13 16:26:29.678451


## Build a dataframe

Check the trait types there are. We'll see that 1) different assets can have different numbers of traits, 2) the same asset can have traits with the same name, i.e. "Celestial Body".

In [3]:
[", ".join(t["trait_type"] for t in a["traits"]) for a in assets][:10]

['Planet, Anomaly, Music, Cockpit, Left Arm, Window, Celestial Body, Celestial Body, Panel 3, Ships, Right Arm, Panel 4, Panel 2',
 'Planet, Cockpit, Left Arm, Right Arm, Window, Celestial Body, Music, Panel 1, Celestial Body',
 'Window, Left Arm, Music, Panel 2, Cockpit, Right Arm, Panel 4, Planet, Celestial Body',
 'Panel 2, Left Arm, Panel 1, Panel 3, Panel 4, Cockpit, Music, Window, Right Arm, Celestial Body',
 'Anomaly, Panel 5, Ships, Cockpit, Right Arm, Celestial Body, Window, Music, Left Arm',
 'Panel 4, Celestial Body, Window, Celestial Body, Right Arm, Cockpit, Music, Panel 5, Left Arm',
 'Panel 1, Panel 4, Cockpit, Right Arm, Ships, Window, Music, Left Arm, Panel 3',
 'Cockpit, Panel 3, Panel 4, Right Arm, Panel 5, Music, Window, Left Arm',
 'Cockpit, Right Arm, Left Arm, Panel 1, Ships, Panel 5, Window, Music',
 'Panel 4, Ships, Right Arm, Planet, Panel 3, Cockpit, Left Arm, Window, Celestial Body, Music, Celestial Body']

All the possible celestial bodies there are:

In [41]:
set([t["value"]  for a in assets for t in a["traits"] if t["trait_type"] == "Celestial Body"])

{'Blue Galaxy',
 'Blue Stars',
 'Green Stars',
 'Large Green Galaxy',
 'Large Purple Galaxy',
 'Purple Galaxy',
 'Purple Stars',
 'Red Stars'}

We will count the number of Celestial Bodies as well as the number of stars and galaxies when tabulating all the information into a dataframe:

(Large vs not large galaxies will be ignored; could be added as some point in the future.)

In [30]:
from typing import Union
from functools import reduce
import operator
import pandas as pd
import numpy as np

pd.set_option("display.max_rows", None)


def turn_assets_into_df_generalized(assets: list) -> Union[pd.DataFrame, list]:

    trait_blueprint = {
        k: "-"  # FIXME store "-" in some variable instead of hardcoding it // also check if there's some nan still somewhere where it shouldn't?
        for k in set([t["trait_type"] for a in assets for t in a["traits"]])
    }
    trait_blueprint["Stars"] = 0
    trait_blueprint["Galaxies"] = 0
    trait_blueprint["Celestial Bodies"] = 0
    del trait_blueprint["Celestial Body"]

    _tt = []
    for a in assets:

        # Calc probability score:
        prob_score = (
            reduce(operator.mul, [t["trait_count"] / 1000 for t in a["traits"]], 1)
            * 1000
        )

        # Get current price:
        price = price_symbol = np.nan
        if a["sell_orders"] is not None:
            so = a["sell_orders"][0]
            price = float(so["current_price"]) / 10 ** int(
                so["payment_token_contract"]["decimals"]
            )
            price_symbol = so["payment_token_contract"]["symbol"]
            # Convert to ETH if necessary:
            if price_symbol not in ["ETH", "WETH"]:
                price *= float(so["payment_token_contract"]["eth_price"])
                price_symbol = "ETH"

        # Get last price:
        last_price = last_price_symbol = last_sale_date = np.nan
        if a["last_sale"] is not None:
            ls = a["last_sale"]
            last_price = float(ls["total_price"]) / 10 ** int(
                ls["payment_token"]["decimals"]
            )
            last_price_symbol = ls["payment_token"]["symbol"]
            # Convert to ETH if necessary:
            if last_price_symbol not in ["ETH", "WETH"]:
                last_price *= float(ls["payment_token"]["eth_price"])
                last_price_symbol = "ETH"
            last_sale_date = ls["event_timestamp"]

        # Traits:
        traits = trait_blueprint.copy()
        for t in a["traits"]:
            if t["trait_type"] == "Celestial Body":
                traits["Celestial Bodies"] += 1
                if "Galaxy" in t["value"]:
                    traits["Galaxies"] += 1
                else:
                    traits["Stars"] += 1
            else:
                if traits[t["trait_type"]] == "-":
                    traits[t["trait_type"]] = t["value"]
                else:
                    traits[t["trait_type"]] += ", " + t["value"]

        # Add everything to the table:
        _tt.append(
            [
                a["token_id"],  # CHANGED
                a[
                    "token_id"
                ],  # FIXME Not really a nice solution // but they don't have names and I need some attribute that is not the index to expose via hover in the Plotly charts.
                price,
                price_symbol,
                last_price,
                last_price_symbol,
                last_sale_date,
                prob_score,
                *[  # All the traits:
                    v for _, v in sorted(traits.items(), key=lambda x: x[0])
                ],
                a["permalink"],
            ]
        )

    # Get the traits names:
    trait_names = [  # All the traits:
        k for k, _ in sorted(trait_blueprint.items(), key=lambda x: x[0])
    ]

    # Turn table into df:
    df = pd.DataFrame(_tt)
    df.columns = [
        "ID",
        "Name",
        "Price",
        "Psymbol",
        "Lastprice",
        "LPsymbol",
        "LPdate",
        "Probscore",
        *trait_names,
        "Link",
    ]
    df["LPdate"] = pd.to_datetime(df.LPdate)
    df = df.set_index("ID").sort_values("Lastprice", ascending=False)

    return df, traits


In [31]:
import numpy as np
import pandas as pd
df, traits = turn_assets_into_df_generalized(assets)
hoverdata = [ "Name", *traits, "Probscore"]

In [32]:
assert set(df.LPsymbol.unique()) == set(["ETH", "WETH", np.nan]), """
This sheet does not do currency conversion at the moment and therefore 
assumes all prices are in (W)ETH. But there are more symbols in the input 
data which would lead to apples being compared to ranges below. Aborting.
"""

## How many pieces have a last price on OpenSea?

In [33]:
print(f"{df[~df.Lastprice.isna()].shape[0] / df.shape[0]:.0%}")

73%


## How are prices evolving over time?

Note that this only takes into account the last sale of each piece.

In [34]:
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (20, 8)
pd.plotting.register_matplotlib_converters()

fig = px.scatter(df, x="LPdate", y="Lastprice", hover_data=hoverdata)
fig.show()

## Price levels today

In [35]:
fig = px.histogram(df, x="Lastprice", marginal="box", hover_data=hoverdata)
fig.show()

## How frequent are the different traits?

In [36]:
pd.set_option("display.max_columns", None)

from IPython.display import display
for trait in sorted(traits.keys()):
    display(
        pd.DataFrame(df[trait].value_counts(normalize=False, sort=True, ascending=True)).transpose()
    )


,Monolith,White Wormhole,Blue Wormhole,Purple Wormhole,Glitch Wormhole,Pink Wormhole,-
Anomaly,7,22,23,24,27,28,869


,5,4,3,2,0,1
Celestial Bodies,1,10,68,223,334,364


,Cat,-,Glyphs,Retro,Dots,Circles,Sound,Chart,Hexagon,Dotface,Planet,Radar,Falcon,Target,Pink
Cockpit,8,21,24,68,70,72,73,76,78,79,81,83,84,89,94


,3,2,1,0
Galaxies,4,32,261,703


,Robo Charge,ETH,Porn,Joint,Jar Shrooms,Watch,Alien Joint,Jar Eyes,Knife,Skull,Hook,Hotdog,8-Ball,Coffee,Whiskey,Takeout,Flag,Syringe,Egg,Jar,Shrooms,Pizza,Booze,Orbit,Banana,Bone,Boba,Burn,Laser,Bong,Wine,Chicken,Goldfish,Zombie,Flower,-,Treasure,V Steering,Buttons,Swipe,Handcrank,Antennae,Slider,Shifter,Big Button,Alien Gearshift,Tablet,Thrust,Hologram,Gearshift,Button Row Slider,Button Row
Left Arm,3,3,3,4,4,5,5,5,5,6,6,7,7,8,8,8,9,9,9,10,10,10,10,11,11,11,12,12,12,13,14,14,14,15,17,21,32,34,38,38,39,41,41,41,42,42,43,43,44,48,49,54


...

,Jar Shrooms,Hotdog,ETH,Porn,Jar Eyes,Joint,Robo Charge,Alien Joint,Skull,Flag,Hook,Pizza,Coffee,Banana,Wine,Bone,Zombie,Egg,Whiskey,Bong,Syringe,Burn,Knife,Watch,Jar,Laser,Chicken,Booze,Goldfish,Shrooms,8-Ball,Orbit,Boba,Takeout,Flower,-,Alien Gearshift,Handcrank,Slider,Thrust,Tablet,Treasure,Button Row Slider,Gearshift,Shifter,V Steering,Hologram,Buttons,Button Row,Swipe,Big Button,Antennae
Right Arm,1,3,4,4,5,5,5,5,6,8,9,9,10,10,10,10,10,10,10,11,11,11,11,12,12,13,13,13,13,14,15,15,16,16,16,21,29,33,33,35,36,36,38,39,40,41,41,44,45,45,49,49


,Battle,UFO,Guardian (rogue),Space Station,Guardian (active),Destroyed Freighter,Flyby,Triforce,Flyby Trio,Cargo,Flyby Dogfight,Freighter Battle,Battle (alt),Fleet,Large Battle,Freighter,Docking,-
Ships,14,15,15,16,18,18,19,20,20,20,21,21,23,23,24,25,26,662


,3,2,1,0
Stars,16,139,396,449


,Eyes,-,Tubes,ETH,Plants,Pink,Green,Blue,Dice,Wires,Crack,White,Screens,Purple,Bolt
Window,20,21,24,30,50,71,71,79,84,86,89,90,93,94,98


,Warp Squad Sixteen,Guardian Marked,RADIOACTIVE,-
special,3,4,13,980


## How much do people pay for the different traits?

In [37]:
for trait in sorted(traits.keys()):

    fig = go.Figure()
    for traitvariant in (
        df.groupby(trait).median().sort_values("Lastprice", ascending=False).index
    ):
        fig.add_trace(
            go.Box(
                y=df[df[trait] == traitvariant].fillna("-").Lastprice.values,
                name=traitvariant,
                boxpoints="all",
                jitter=0.2,
                whiskerwidth=0.2,
                marker_size=2,
                line_width=1,
            )
        )
    fig.update_layout(title=trait)
    fig.show()

**Some Observations**:

- The monolith anomaly commands much higher prices than other anomalies. It is also the most seldom anomaly.
- The more celestial bodies, the higher the price. Same with galaxies and stars. (4 stars is really seldom and outperforms 0-3 stars massively.)
- A cat cockpit slightly outperforms other cockpits. Interestingly, glyphs perform average, even though they are much rarer than average.
- Arms: Joints perform best, followed by ETH, bong, porn, alien joint, and skull. Flag is clearly worst (I wonder why?). Bongs perform much better than their rarity would suggest, whereas flags left perform much worse than their rarity would suggest (not so much for flags right, strangely).
- Music doesn't make a difference.
- Panels: Crypto, bounty, tetris, spam are the best preformers which is very much in line with their rarity.
- Planet: Anger orb and destroyed planet are clearly performing best and also rarest.
- Ships: Doesn't make much of a difference.
- Window: Doesn't make much of a difference.
- Special: Specials in general add to the value. Warp squad sixteen performs very high.

## What is pricier, galaxies or stars?

There seems to be no fundamental difference between stars and galaxies. In either case, rarity is the differentiator: 3 galaxies is slightly rare, 4 stars is rare, 5 bodies overall is rare and 6 bodies is superrare – so these all command higher prices.

In [ ]:
df["Celestialstype"] = df.apply(
    lambda x: "Starsonly"
    if x["Galaxies"] == 0
    else "Galaxiesonly"
    if x["Stars"] == 0
    else "Both",
    axis=1,
)
px.box(df, x="Celestial Bodies", y="Lastprice", color="Celestialstype", points="all", hover_data=hoverdata)


## Probscore as a simple unified metric to predict prices

The Probscore metric is simply the product of the probabilities of the different traits of a piece. There is currently not much value in that metric. While only the lowest probabilities are able to command the very high outlier prices, overall correlation between Probscore and Lastprice remains low:

In [ ]:
fig = px.scatter(df, x="Probscore", y="Lastprice", hover_data=hoverdata)
fig.show()
df[["Probscore", "Lastprice"]].corr()

## What to buy?

In [ ]:
fig = px.scatter(df[~df.Price.isna()], x="Probscore", y="Price", hover_data=hoverdata)
fig.show()
df[["Probscore", "Price"]].corr()

In [ ]:
# FIXME #84 is shown with a price even though it doesn't have one on OpenSea.

In [ ]:
df[df.Name == "7350"]